# Relation between continuous-time and discrete-time poles

This notebook allows you to explore the relation between continuous-time and discrete-time pole locations.

In [ ]:
# load required packages - make sure these are installed in your Julia environment
using Interact, Plots
using ControlSystems

In [ ]:
# helper functions
# create a sample system with given continuous-time poles & its discretized variant
function createSystems(cPoles, Δt)
    cSys = zpk([], cPoles, 1.0)   # continuous-time system defined by its zeros, poles, and gain
    dSys = c2d(cSys, Δt)          # convert continuous-time system to discrete-time using given sampling time Δt
    return cSys, dSys
end
# construct the poles we will visualize - either a single real pole, or a conjugate pair, depending on the supplied real and imaginary values
function parsePoles(RePart, ImPart)
    cPoles = Complex{Float64}[]
    p = RePart + ImPart*im          # construct pole from given values
    push!(cPoles, p) 
    if imag(p) != 0.0 
        push!(cPoles, conj(p))      # also add its conjugate pair if the pole is imaginary
    end
    return cPoles
end;

The following interactive plots allows you to explore how the location of poles in continuous time and the discretization time step Δt affects the pole locations of the resulting discrete-time system. In particular, explore the following:
1. How does the real part of continuous-time poles affect the location of the discrete-time poles? What do more stable poles correspond to?
2. How does the imaginary part of continuous-time poles affect the location of the discrete-time poles? Where are the more oscillatory poles mapped to?
3. What happens to the range of discrete-time poles the continuous-time ones are mapped to as Δt is decreased? How does Δt influence the location of the poles?

In [ ]:
# define limits & step size for the range of real and imaginary values of the poles we will explore
maxRe = 2.0; maxIm = 3.0; pStep = 0.1
# also limits for discretization step Δt
minΔt = 0.001; maxΔt = 1.0; ΔtStep = 0.01

# interactive visualization of pole locations depending on discretization step
@manipulate for Δt = widget(minΔt:ΔtStep:maxΔt, label="Δt"), RePart = widget(-maxRe:pStep:maxRe, label="Real part"), ImPart = widget(0.0:pStep:maxIm, label="Imaginary part")
    # construct discrete & continuous time system with specified poles
    cPoles = parsePoles(RePart, ImPart)
    cSys, dSys = createSystems(cPoles, Δt)
    dPoles = pole(dSys)
    
    # plot continuous-time pole locations
    p1 = scatter(xlims=(-maxRe-0.1, maxRe+0.1), ylims=(-maxIm-0.1, maxIm+0.1), label="", title="Continuous-time poles", xlabel="Re", yLabel="Im")
    [scatter!(p1, [real(cPoles[i])], [imag(cPoles[i])], label="", msize=8, shape=:x) for i in eachindex(cPoles)]
    # shade left hand plane - region of stable poles
    plot!(Shape(-maxRe .+ [0,maxRe,maxRe,0], -maxIm .+ [0,0,2*maxIm,2*maxIm]), opacity=.2, color=:green, label="")
    
    # plot step response for continuous-time system
    cy, ct, cx = step(cSys, 20.0)
    p2 = plot(ct, cy, linewidth=3, label="", title="Continuous-time step response", xlabel="t", ylabel="y")
    
    # plot discrete-time pole locations
    p3 = scatter(xlims=(-maxRe-0.1, maxRe+0.1), ylims=(-maxIm-0.1, maxIm+0.1), label="", title="Discrete-time poles", xlabel="Re", yLabel="Im")
    [scatter!(p3, [real(dPoles[i])], [imag(dPoles[i])], label="", msize=8, shape=:x) for i in eachindex(dPoles)]
    # plot unit circle - region of stable poles
    plot!(p3, (x->cos(x)), (x->sin(x)), 0, 2π, line=2, color=:green, fill = (0, 0.2, :green), lab="")
    
    # plot step response for discrete-time system
    dy, dt, dx = step(dSys, 20.0)
    p4 = plot(dt, dy, linewidth=3, linetype=:steppost, label="", title="Discrete-time step response", xlabel="t", ylabel="y")
    
    # assemble plots in nice layout
    vbox(
        plot(p1, p2, p3, p4, size=(1000, 600), layout=@layout [a b; c d]))
end

Observe that the pole at $z = 0$ does not have an equivalent continuous-time counterpart. 
1. What type of continuous-time poles become mapped close to the $z = 0$ location?
2. Using the following interactive plot, see how the system's step response (and the evolution of the states) changes with the $P$ number of poles at $z = 0$. Can such a behavior be reproduced by a continuous-time system? Why, or why not?

In [ ]:
# interactive plots visualizing the step response of a discrete-time system with P poles at z = 0
@manipulate for P = widget(1:9, label="P")
    # construct discrete-time system with Δt = 1.0 and P poles at z = 0
    dPoles = zeros(P)
    dSys = zpk([], dPoles, 1.0, 1.0)
    # plot the step response of the output and states
    dy, dt, dx = step(dSys, 10.0)
    p1 = plot(dt, dy, linewidth=3, label="", linetype=:steppost, title="Step response (output)", xlabel="t", ylabel="y")
    p2 = plot(dt, dx, linewidth=3, label="", linetype=:steppost, title="Step response (states)", xlabel="t", ylabel="x")
    vbox(
        plot(p1, p2, size=(1000, 400), layout=@layout [a b]))
end